In [1]:
import numpy as np
import pandas as pd

samsung = np.load("samsung.npy")
kospi200 = np.load("kospi200.npy")

print(samsung)
print(kospi200)

##########################################################################################
               
def split_xy(dataset, n_steps, y_column):
    x,y = list(), list()
    for i in range(len(dataset)):     
        x_end_number = i + n_steps
        y_end_number = x_end_number + y_column
        if y_end_number > len(dataset):  
            break
            
        tmp_x, tmp_y = dataset[i:x_end_number, : ], dataset[x_end_number:y_end_number, 3] 
        x.append(tmp_x)                                      
        y.append(tmp_y)
    return np.array(x), np.array(y)

x, y = split_xy(samsung,5,1)
print(x[0,:], '\n',y[0])

[[   53000    53900    51800    51900 39565391]
 [   52600    53200    51900    52600 23104720]
 [   52600    52800    50900    50900 16128305]
 ...
 [   59100    59700    58800    59100 16446102]
 [   58800    58800    56800    57200 20821939]
 [   57800    58400    56400    56400 19749457]]
[[   320.56    321.      316.75    316.75 173911.  ]
 [   317.79    319.53    314.95    315.87 141252.  ]
 [   315.93    316.08    312.6     314.42 123610.  ]
 ...
 [   294.38    295.67    292.45    293.98  85731.  ]
 [   293.27    294.11    287.09    288.37 101535.  ]
 [   290.24    291.47    284.53    284.53 101455.  ]]
[[   53000    53900    51800    51900 39565391]
 [   52600    53200    51900    52600 23104720]
 [   52600    52800    50900    50900 16128305]
 [   51700    51700    50600    51600 13905263]
 [   52000    52200    51200    51300 10314997]] 
 [50100]


In [2]:
print(x.shape, y.shape)

(421, 5, 5) (421, 1)


In [14]:
# 데이터셋 나누기
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, 
                                                   random_state=1,
                                                    test_size=0.6,
                                                    shuffle=False)

print(x_train, x_test)
print(y_train, y_test)

(253, 5, 5)

In [4]:
# 데이터 전처리 StandardScaler
from sklearn.preprocessing import StandardScaler

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1] * x_train.shape[2]))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1] * x_test.shape[2]))
                          
x_train.shape

(168, 25)

In [5]:
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train) 
x_test = scaler.transform(x_test)

In [6]:
x_train = np.reshape(x_train, (x_train.shape[0], 5, 5))
x_test = np.reshape(x_test, (x_test.shape[0], 5, 5))

In [7]:
# 2. 모델 구성
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization, Dropout, Reshape

model = Sequential()

model.add(LSTM(32, input_shape=(5,5), activation='tanh'))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Using TensorFlow backend.
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\d

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                4864      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                1088      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 520       
________________________________________________

In [8]:
#3. 훈련
model.compile(loss='mse', optimizer='adam', metrics=['mae'])
model.fit(x_train, y_train, epochs=100, batch_size=1, validation_split=0.2,verbose=0)
# validation_split=0.2 == 트레인 데이터에서 0.2 만큼을 검증용으로 사용하겠다.

Instructions for updating:
Use tf.cast instead.


In [9]:
#4. 평가예측
evaluation = model.evaluate(x_test, y_test, batch_size=1)
print("evaluation:{}".format(evaluation))

253/253 [==============================] - 0s 623us/step
evaluation:[2316034769.4545455, 47897.8125]


In [15]:
# RMSE 만들기
from sklearn.metrics import mean_squared_error
y_predict = model.predict(x_test, batch_size=1)


def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test
                                      , y_predict))
print("RMSE :", RMSE(y_test, y_predict))

# R2 지표 만들기
from sklearn.metrics import r2_score

r2_y_predict  = r2_score(y_test, y_predict)
print("R2 : ", r2_y_predict)

RMSE : 48125.210803873546
R2 :  -105.0743070873162


In [11]:
for i in range(5):
    print("종가(test) : {}".format(y_test[i]))
    print("종가(pred) : {}".format(y_predict[i]))

종가(test) : [42750]
종가(pred) : [0.979832]
종가(test) : [42150]
종가(pred) : [0.98752314]
종가(test) : [42000]
종가(pred) : [0.99116427]
종가(test) : [43050]
종가(pred) : [0.9907415]
종가(test) : [44750]
종가(pred) : [0.9937941]
